In [ ]:
# IMPORT spikeinterface and probeinterface
import spikeinterface.full as si
import probeinterface as pi

# IMPORT other libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from pathlib import Path

In [ ]:
# SET the file paths
baseFolder         = Path('.')
recordingFile      = baseFolder / 'HSW_2023_06_22__09_17_51__05min_43sec__hsamp_256ch_25000sps.bin'

# SET job params for parallel processing steps (n_jobs = nCores-1)
jobKwArgs          = dict(n_jobs         = 5,                          
                          chunk_duration = "1s",
                          progress_bar   = True,
                          mp_context     = "spawn")

In [ ]:
# BIN file parameeters
numChannels        = 256
samplingFrequency  = 25000
gainToMicroVolts   = 6.25e3/32768
offsetToMicroVolts = 0
dataType           = 'int16'
timeAxis           = 0
fileOffset         = 8

# LOAD BIN file in 'lazy' manner - access as required
data               = si.read_binary(recordingFile,
                                    num_chan           = numChannels,
                                    sampling_frequency = samplingFrequency,
                                    dtype              = dataType,
                                    gain_to_uV         = gainToMicroVolts,
                                    offset_to_uV       = offsetToMicroVolts, 
                                    time_axis          = timeAxis,
                                    file_offset        = fileOffset,
                                    is_filtered        = False)

In [ ]:
# LOAD fma32 probe
probe         = pi.io.read_probeinterface('cocoFma256.json')

# SET probe into recording -  this will reorder the probe contact_ids based on device_channel_ids
# SEE answer to query - https://github.com/SpikeInterface/spikeinterface/issues/246
dataWithProbe = data.set_probegroup(probe, group_mode = "by_shank")

# LOAD fma256SpecsFlat - ordered by ascending fma number (0:255, sets of 32)
fma256Specs   = pd.read_csv('cocoFma256.csv')

# REORDER fma256Specs by ascending hsw number
fma256Specs   = fma256Specs.sort_values(by=['hsw'])

# ADD fmaID to properties 
dataWithProbe.set_property(key = 'fmaID', values = fma256Specs['fmaID'].values)

In [ ]:
# HIGHPASS filter the data
dataWithProbeFiltered       = si.filter(dataWithProbe,
                                        band         = 250,
                                        btype        = 'highpass',
                                        ftype        = 'butter',
                                        filter_order = 4)

In [ ]:
# Start timer
startTime = time.time()

# SAVE highpass data into folder
if (baseFolder / "preprocessed").is_dir():
    preprocessedData = si.load_extractor(baseFolder / "preprocessed")
else:
    preprocessedData = dataWithProbeFiltered.save(folder = baseFolder / "preprocessed",
                                                  **jobKwArgs)
    
# End timer
endTime = time.time()

# Calculate elapsed time
elapsedTime = endTime - startTime
print("Elapsed time: ", elapsedTime)     

In [ ]:
# Start timer
startTime = time.time()

# SORT the preprocessed data using spykingcircus called through docker image
sortedData = si.run_sorter("mountainsort4", preprocessedData, 
                           output_folder     = baseFolder / "sorted",                        
                           docker_image      = True, 
                           verbose           = True,
                           detect_sign       = -1,
                           detect_threshold  = 3,
                           detect_interval   = 30,
                           filter            = False,
                           whiten            = True,
                           num_workers       = 6,
                           adjacency_radius  = 50,
                           clip_size         = 40)

# End timer
endTime = time.time()

# Calculate elapsed time
elapsedTime = endTime - startTime
print("Elapsed time: ", elapsedTime) 

In [ ]:
# LOAD the preprocessed and sorted data separately
recording = si.load_extractor(baseFolder / "preprocessed")
sorting   = sortedData
sorting

In [ ]:
waveforms = si.extract_waveforms(recording, sorting,
                                 folder                  = baseFolder / "waveformAll", 
                                 max_spikes_per_unit     = None,
                                 ms_before               = 0.4,
                                 ms_after                = 1.6,
                                 sparse                  = True,
                                 num_spikes_for_sparsity = 100,
                                 method                  = "radius",
                                 radius_um               = 50,
                                 **jobKwArgs)

In [ ]:
si.export_to_phy(waveforms,
                 output_folder = baseFolder / 'curation', 
                 **jobKwArgs)